# Second Draft Of Credit Model - Auto Rejection and Naive Loan + Monthly Interest

### The second draft of this model will be comprised of two separate models, an auto rejector and a naive loan and monthly interest reccomender.

### Parameters:
(Auto Rejection)
- Age of company
- Growth%
- More parameters TBD (waiting on Ana's Superset query)

(Naive Loan + Interest)
- Age of company
- Initial reported revenue 

### Outputs:
(Auto Rejection)
- True/False rejection

(Naive Loan + Interest)
- Initial loan amount offer (gross_value)
- Monthly interest offer

### Data Used:
(Auto Rejection)
- Background checker data from Ana's Superset Query 
- Accounts.json taken from HTTPS Postman get request

(Naive Loan + Interest)
- Query of current clients via superset
- Gian's monthly growth calculations

### Helpful People:
- Hugo, Will, Well, Vini S, Vini R, Vitor, Gian, Ana, Caruba, Prando, Todos!

## General Data Cleaning and Imports

In [1]:
import pandas as pd
import numpy as np
import json

#Loading and cleaning the Auto Rejection data
json_file_path = "/Users/alizia/AliDivi/proof-of-concept/datasets/dummyDataFinal.json"
bc_accounts = pd.read_json(json_file_path)

json_file_path = "insert Ana's superset query on rejected clients here"
#bc_accounts_with_rejections = pd.read_json(json_file_path)

#Loading and cleaning the Naive Loan/Interest data
json_file_path = "/Users/alizia/AliDivi/proof-of-concept/datasets/clients_given_offer.json"
clients_given_offer = pd.read_json(json_file_path)
clients_given_offer

,account_nickname,cnpj,age,initial_reported_revenue,rating,gross_value,monthly_interest_rate
0,100 Foods,27997623000137,6.438131,93069.00,B,10367.31,2.00
1,1Recado,34424334000104,3.968539,14045.92,B,10367.42,2.00
2,4k Cosmetics,33113626000163,4.340894,24751.71,CC,2113.86,2.65
3,ADVBOX,30750260000145,5.093819,193023.15,BBB,31280.26,2.00
4,AGENCIA LEAD,31410323000187,4.915855,19673.92,CC,5183.83,2.50
...,...,...,...,...,...,...,...
109,YOUBECOME,25464755000178,6.950120,77415.00,CC,5234.15,2.75
110,Yes We Grow,26624044000186,6.648950,378691.98,BBB,105880.59,1.65
111,Yori Cosméticos e Beleza,32615057000191,4.480527,10456.62,CC,2080.00,2.90
112,iKEG,29973569000115,5.348444,717750.56,BBB,45450.00,2.00


## Auto Rejection Model and Evaluation

Currently waiting on Ana's query of all companies Divi has evaluated.

## Naive Loan + Interest Models and Evaluation

In [2]:
clients_given_offer
clients_given_offer = clients_given_offer.rename(columns={"initial_reported_revenue_x": "initial_reported_revenue"})


In [3]:
naive_data = clients_given_offer[['account_nickname', 'cnpj', 'age', 'initial_reported_revenue', 'gross_value', 'monthly_interest_rate']].copy()
naive_data

,account_nickname,cnpj,age,initial_reported_revenue,gross_value,monthly_interest_rate
0,100 Foods,27997623000137,6.438131,93069.00,10367.31,2.00
1,1Recado,34424334000104,3.968539,14045.92,10367.42,2.00
2,4k Cosmetics,33113626000163,4.340894,24751.71,2113.86,2.65
3,ADVBOX,30750260000145,5.093819,193023.15,31280.26,2.00
4,AGENCIA LEAD,31410323000187,4.915855,19673.92,5183.83,2.50
...,...,...,...,...,...,...
109,YOUBECOME,25464755000178,6.950120,77415.00,5234.15,2.75
110,Yes We Grow,26624044000186,6.648950,378691.98,105880.59,1.65
111,Yori Cosméticos e Beleza,32615057000191,4.480527,10456.62,2080.00,2.90
112,iKEG,29973569000115,5.348444,717750.56,45450.00,2.00


In [4]:
X = naive_data[['age', 'initial_reported_revenue']]
y = naive_data[['gross_value', 'monthly_interest_rate']]

### K Neighbors

In [5]:
def naive_kneighbors():
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.model_selection import train_test_split
    model = KNeighborsRegressor()
    X = naive_data[['age', 'initial_reported_revenue']]
    y = naive_data[['gross_value', 'monthly_interest_rate']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=100)
    model.fit(X_train,y_train)
    print(model.score(X_test,y_test))


In [6]:
naive_kneighbors()

0.3999203566872057


### LinearRegressor

In [7]:
def naive_linreg():
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    model = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=100)
    model.fit(X_train,y_train)
    print(model.score(X_test,y_test))

In [8]:
naive_linreg()

0.28300424080807296


So far, these scores don't seem too great. Let's use MultiOutputRegressor to see if these parameters are mostly independent of each other and not good indicators of a solid model.

### KNeighbors Eval

In [9]:
def kneighbors_approach_eval(X, y):
    # example of evaluating direct multioutput regression with an SVM model
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.model_selection import train_test_split
    from sklearn.multioutput import MultiOutputRegressor
    from sklearn.model_selection import RepeatedKFold
    from sklearn.model_selection import cross_val_score
    from numpy import absolute
    from numpy import mean
    from numpy import std

    model = KNeighborsRegressor()
    wrapper = MultiOutputRegressor(model)
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    n_scores = cross_val_score(wrapper, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    n_scores = absolute(n_scores)
    print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


In [10]:
kneighbors_approach_eval(X, y)

MAE: 54376.777 (31369.674)


In [11]:
def corr_matrix_kneighbors():
    corr_matrix = naive_data.drop(columns=["account_nickname", "cnpj"]).corr()
    print(corr_matrix)

corr_matrix_kneighbors()

                               age  initial_reported_revenue  gross_value  \
age                       1.000000                  0.106551     0.099923   
initial_reported_revenue  0.106551                  1.000000     0.722570   
gross_value               0.099923                  0.722570     1.000000   
monthly_interest_rate    -0.310012                 -0.260995    -0.326858   

                          monthly_interest_rate  
age                                   -0.310012  
initial_reported_revenue              -0.260995  
gross_value                           -0.326858  
monthly_interest_rate                  1.000000  


Something went wrong here. Either the model is very bad or my evaluation is broken.

## The Issue and Solutions

One clear issue is the lack of rows in the data set. No matter how many columns we add, there just arent enough rows for training for us to say that each form of regression is good or bad yet. Waiting for more data from Prando/Ana

### In the meantime...

Let's try hyper parameters for the models that might be useful to the naive loan + monthly interest model.

In [12]:
def naive_linreg_v2():
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    model = LinearRegression()
    X = naive_data[['age', 'initial_reported_revenue']]
    y = naive_data[['gross_value', 'monthly_interest_rate']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=100)
    model.fit(X_train,y_train)
    print(model.score(X_test,y_test))
naive_linreg_v2()

0.28300424080807296


We can't hyper parametrize lin reg, so let's move onto k neighbors

In [13]:
def hyperparameters():
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import GridSearchCV

    #List Hyperparameters that we want to tune.
    leaf_size = list(range(1,50))
    n_neighbors = list(range(1,30))
    p=[1,2]
    #Convert to dictionary
    hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
    #Create new KNN object
    knn_2 = KNeighborsRegressor()
    #Use GridSearch
    clf = GridSearchCV(knn_2, hyperparameters, cv=10)
    #Fit the model
    best_model = clf.fit(X,y)
    #Print The value of best Hyperparameters
    print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
    print('Best p:', best_model.best_estimator_.get_params()['p'])
    print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])


This also does not work :/

The next best thing would be to do some feature engineering (switching up the inputs and outputs in hopes for better scores). Let's start with simplifying the output of Lin Reg:

In [14]:
def naive_linreg_v2_simpler():
    naive_data_copy = naive_data
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    model = LinearRegression()
    X = naive_data_copy[['age', 'initial_reported_revenue']]
    y = naive_data_copy[['gross_value']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=100)
    model.fit(X_train,y_train)
    print(model.score(X_test,y_test))
naive_linreg_v2_simpler()

0.44636301469802286


#### A lot better! Let's see if we can add more parameters to the inputs too.

In [31]:

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
model = LinearRegression()
json_file_path = "/Users/alizia/AliDivi/proof-of-concept/datasets/clients_given_offer.json"
clients_given_offer = pd.read_json(json_file_path)
# clients_given_offer['rating'] = clients_given_offer['rating'].rank(method='dense', ascending=False).astype(int)
clients_given_offer
# print(model.score(X_test,y_test))


,account_nickname,cnpj,age,initial_reported_revenue,rating,gross_value,monthly_interest_rate
0,100 Foods,27997623000137,6.438136,93069.00,B,10367.31,2.00
1,1Recado,34424334000104,3.968544,14045.92,B,10367.42,2.00
2,4k Cosmetics,33113626000163,4.340899,24751.71,CC,2113.86,2.65
3,ADVBOX,30750260000145,5.093824,193023.15,BBB,31280.26,2.00
4,AGENCIA LEAD,31410323000187,4.915860,19673.92,CC,5183.83,2.50
...,...,...,...,...,...,...,...
109,YOUBECOME,25464755000178,6.950125,77415.00,CC,5234.15,2.75
110,Yes We Grow,26624044000186,6.648955,378691.98,BBB,105880.59,1.65
111,Yori Cosméticos e Beleza,32615057000191,4.480533,10456.62,CC,2080.00,2.90
112,iKEG,29973569000115,5.348449,717750.56,BBB,45450.00,2.00


In [16]:
X = clients_given_offer[['age', 'initial_reported_revenue','rating']]
y = clients_given_offer[['gross_value']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=100)
model.fit(X_train,y_train)
print(model.score(X_test,y_test))

0.4331487718218039


In [17]:
X = clients_given_offer[['age', 'initial_reported_revenue','rating']]
y = clients_given_offer[['gross_value', 'monthly_interest_rate']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=100)
model.fit(X_train,y_train)
print(model.score(X_test,y_test))

0.4633029679145447


Adding credit rating alongside monthy interest rate seems to yield the most accurate results so far, but we can see that even with feature engineering, the score is not changing by much. Perhaps a different regressor is better. I will discuss this with Vini in our upcoming meeting